In [1]:
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [2]:
"""
Minimal setup for a tiny LLaMA model with randomly initialized weights.

Run with:
    python examples/minimal_llama_setup.py
"""

import os
os.environ["XLA_FLAGS"]="--xla_force_host_platform_device_count=8"


import jax
import jax.numpy as jnp

from models.llama.config import ModelConfig
from models.llama.model import LLaMa
from utils.kvcache import KVCache
from utils.ops import build_attn_mask

# Tiny config so it runs fast on CPU
cfg = ModelConfig(
    vocab_size=128,
    dim=32,
    ffn_hidden_dim=64,
    n_layers=2,
    n_heads=4,
    n_kv_heads=2,
    activation_fn="silu",
    max_seqlen=16,
    rope_theta=10000.0,
    rms_norm_eps=1e-5,
    dtype=jnp.float32,
    use_scaled_rope=False,
)

model = LLaMa(cfg)

# Dummy token batch
tokens = jnp.array(
    [
        [1, 5, 7, 9],
        [4, 3, 2, 0],
    ],
    dtype=jnp.int32,
)
true_lengths = jnp.array([4, 3], dtype=jnp.int32)  # second sequence is padded at the end
bsz, seqlen = tokens.shape

# Fresh KV cache and mask
kv_cache = KVCache.new(
    n_layers=cfg.n_layers,
    bsz=bsz,
    max_seqlen=cfg.max_seqlen,
    kv_heads=cfg.n_kv_heads,
    head_dim=cfg.head_dim,
    dtype=cfg.dtype,
)
mask = build_attn_mask(seqlen, kv_cache, true_lengths)

# Initialize random weights, then run a forward pass
rng = jax.random.PRNGKey(0)
variables = model.init(rng, tokens, true_lengths, kv_cache, mask)
# logits, updated_cache = model.apply(variables, tokens, true_lengths, kv_cache, mask)

# print("Logits shape:", logits.shape)  # (batch, seq, vocab_size)
# print("Updated cache positions:", updated_cache.seq_positions)


TypeError: LLaMa.__init__() missing 1 required positional argument: 'sharding'

In [9]:
from typing import Optional
from jax.sharding import NamedSharding,PartitionSpec as PS,Mesh
default_mesh = jax.make_mesh((4,), ('x',))

def mesh_sharding(
    pspec: PS, mesh: Optional[Mesh] = None,
  ) -> NamedSharding:
  if mesh is None:
    mesh = default_mesh
  return NamedSharding(mesh, pspec)
x = jax.device_put(jnp.arange(16).reshape(4,4),mesh_sharding(PS('x',)))


In [11]:
jax.debug.visualize_array_sharding(x)

┌───────────────────────┐
│         CPU 0         │
├───────────────────────┤
│         CPU 1         │
├───────────────────────┤
│         CPU 2         │
├───────────────────────┤
│         CPU 3         │
└───────────────────────┘

In [10]:
print(jax.device_count())

8


In [5]:
model = LLaMa(cfg,mesh_sharding(PS('x','y')))

In [6]:
model

LLaMa(
    # attributes
    args = ModelConfig(vocab_size=128, dim=32, ffn_hidden_dim=64, n_layers=2, n_heads=4, n_kv_heads=2, activation_fn='silu', max_seqlen=16, rope_theta=10000.0, rms_norm_eps=1e-05, mode='inference', dtype=<class 'jax.numpy.float32'>, use_scaled_rope=False)
    sharding = NamedSharding(mesh=Mesh('x': 4, 'y': 2, axis_types=(Auto, Auto)), spec=PartitionSpec('x', 'y'), memory_kind=device)
)

AttributeError: "Embed" object has no attribute "shape".